# 한국어 금융보안 데이터셋 번역 작업

## 작업 개요
이 노트북은 영어/중국어로 작성된 금융보안 관련 QA 데이터셋을 한국어로 번역하는 작업을 수행합니다.

## 대상 파일들
1. **CyberMetric/mcqa_org.csv** - 영어 전용 MCQA 데이터
2. **SecBench/mcqa_org.csv** - 영어/중국어 혼용 MCQA 데이터  
3. **SecBench/qa_org.csv** - 영어/중국어 혼용 QA 데이터

## 작업 흐름
1. 각 `_org.csv` 파일을 읽어서 데이터 구조 및 언어 분포 확인
2. **언어 자동 감지**: SecBench 데이터의 영어/중국어 혼용 상황 처리
3. 로컬 LLM을 사용하여 한국어로 번역
4. 번역된 결과를 원래 폴더에 한국어 버전으로 저장
5. 번역 품질 검증

## 주요 개선사항
- **언어 자동 감지 기능**: SecBench 데이터의 영어/중국어 혼용 문제 해결
- **개별 항목별 언어 판단**: 질문과 선택지마다 독립적으로 언어 감지
- **이미 번역된 내용 건너뛰기**: 한국어 텍스트는 재번역하지 않음

In [1]:
# 필요한 라이브러리 설치 및 설정

# !pip install -q pandas tqdm 
# !pip install -q transformers==4.55.0 # llm requires >=4.46.0
# !pip install -q safetensors==0.4.3 # downgrade for torch 2.1.0
# !pip install -q bitsandbytes==0.43.2 accelerate==1.9.0 # quantization

print("라이브러리 설치 완료!")

import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import re
from typing import List, Dict, Tuple
import time

라이브러리 설치 완료!


In [2]:
# LLM 모델 로드 및 번역 함수 정의

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import unicodedata

def load_translation_model(model_name="microsoft/DialoGPT-medium"):
    """번역용 LLM 모델을 로드합니다."""
    try:
        # 양자화 설정 (메모리 절약)
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )
        
        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            padding_side="left"  # Decoder-only 모델용 left padding
        )
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quantization_config,
            device_map="auto",
            torch_dtype=torch.float16,
            attn_implementation="eager"  # SDPA 요구사항 우회
        )
        
        # pad_token 설정
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            
        print(f"모델 '{model_name}' 로드 완료!")
        return tokenizer, model
        
    except Exception as e:
        print(f"모델 로드 실패: {e}")
        print("대안: OpenAI API 또는 다른 번역 서비스를 사용하세요.")
        return None, None

def detect_language(text: str) -> str:
    """텍스트의 언어를 자동 감지합니다."""
    
    # 한국어는 이미 번역 완료된 것으로 간주
    korean_chars = sum(1 for char in text if '\uac00' <= char <= '\ud7af')
    if korean_chars > len(text) * 0.1:  # 10% 이상이 한글이면 한국어
        return "ko"
    
    # 중국어 문자 감지 (간체/번체 포함)
    chinese_chars = sum(1 for char in text if '\u4e00' <= char <= '\u9fff')
    
    # 영어 문자 감지 (알파벳)
    english_chars = sum(1 for char in text if char.isascii() and char.isalpha())
    
    # 전체 텍스트 길이
    total_chars = len(text.replace(' ', '').replace('\n', ''))
    
    if total_chars == 0:
        return "unknown"
    
    # 중국어 비율이 높으면 중국어
    if chinese_chars > total_chars * 0.3:
        return "zh"
    
    # 영어 비율이 높으면 영어
    if english_chars > total_chars * 0.5:
        return "en"
    
    # 혼용되어 있는 경우 더 많은 쪽으로 판단
    if chinese_chars > english_chars:
        return "zh"
    elif english_chars > chinese_chars:
        return "en"
    else:
        return "unknown"

def clean_translation_result(text: str) -> str:
    """번역 결과에서 불필요한 요소들을 제거합니다."""
    # 프롬프트 잔여물 및 메타 설명 제거
    unwanted_patterns = [
        r"English:\s*", r"Korean:\s*", r"Chinese:\s*", r"Korean translation:\s*",
        r"---+", r"\*\*.*?\*\*", r"Final Answer?:?",
        r"번역:\s*", r"답변:\s*", r"결과:\s*",
        r"The original.*?requested\.", r"However.*?ask!",
        r"I will follow.*?format\.", r"The translation.*?style\.",
        r"The provided text.*?terminology\.", r"If you need.*?ask!",
        r"Translate.*?Korean", r"Translation.*?format"
    ]
    
    for pattern in unwanted_patterns:
        text = re.sub(pattern, "", text, flags=re.IGNORECASE | re.DOTALL)
    
    # 음성 번역 패턴 제거 (한국어 + 괄호 안 영어 발음)
    # 예: "정보 보증 (Jeongbo bojung)" → "정보 보증"
    text = re.sub(r'\s*\([A-Za-z\s-]+\)', '', text)
    
    # 영어 설명문 제거 (긴 영어 문장들)
    sentences = text.split('.')
    cleaned_sentences = []
    for sentence in sentences:
        sentence = sentence.strip()
        if sentence:
            # 영어 비율이 높은 문장 제거
            english_chars = sum(1 for char in sentence if char.isascii() and char.isalpha())
            total_chars = len(sentence.replace(' ', ''))
            if total_chars > 0 and english_chars / total_chars < 0.7:  # 영어 비율 70% 미만만 유지
                cleaned_sentences.append(sentence)
    
    text = '. '.join(cleaned_sentences)
    
    # 연속된 공백과 줄바꿈 정리
    text = re.sub(r'\n\s*\n', '\n', text)  # 빈 줄 제거
    text = re.sub(r'\s+', ' ', text)  # 연속 공백을 하나로
    text = text.strip()
    
    return text

def translate_text(text: str, tokenizer, model, source_lang: str = "auto", target_lang: str = "ko") -> str:
    """텍스트를 한국어로 번역합니다. source_lang이 'auto'면 자동 감지합니다."""
    
    # 언어 자동 감지
    if source_lang == "auto":
        detected_lang = detect_language(text)
        if detected_lang == "ko":
            return text  # 이미 한국어면 번역하지 않음
        elif detected_lang == "unknown":
            # 알 수 없는 경우 영어로 가정
            detected_lang = "en"
        source_lang = detected_lang
    
    # 의미 번역을 강조하는 프롬프트 (음성 번역 방지)
    if source_lang == "en":
        prompt = f"Translate this English text to Korean with proper meaning (not pronunciation). Use standard Korean terminology:\n\n{text}\n\nKorean translation:"
    elif source_lang == "zh":
        prompt = f"Translate this Chinese text to Korean with proper meaning (not pronunciation). Use standard Korean terminology:\n\n{text}\n\nKorean translation:"
    else:
        prompt = f"Translate to Korean with proper meaning (not pronunciation):\n\n{text}\n\nKorean translation:"
    
    try:
        # 토큰화 (attention_mask 포함)
        inputs = tokenizer(
            prompt, 
            return_tensors="pt", 
            truncation=True, 
            max_length=512,
            padding=True
        )
        
        # 입력 텐서를 모델과 같은 디바이스로 이동
        device = next(model.parameters()).device
        inputs = {key: value.to(device) for key, value in inputs.items()}
        
        # 생성 (GPU 최적화된 파라미터)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,  # input_ids와 attention_mask 모두 전달
                max_new_tokens=100,  # 더 짧게 (속도 향상)
                num_return_sequences=1,
                temperature=0.2,  # 더 결정적으로 (속도 향상)
                top_p=0.9,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.1,  # 반복 방지
                use_cache=True  # 캐시 사용으로 속도 향상
            )
        
        # 디코딩
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 번역 결과 추출 (개선된 로직)
        if "Korean translation:" in generated_text:
            translated = generated_text.split("Korean translation:")[-1].strip()
        elif "Korean:" in generated_text:
            translated = generated_text.split("Korean:")[-1].strip()
        else:
            # 프롬프트 제거
            translated = generated_text[len(prompt):].strip()
        
        # 결과 정제
        translated = clean_translation_result(translated)
        
        # 영어 비율이 너무 높으면 원문 반환 (번역 실패로 간주)
        if translated:
            english_chars = sum(1 for char in translated if char.isascii() and char.isalpha())
            korean_chars = sum(1 for char in translated if '\uac00' <= char <= '\ud7af')
            total_chars = len(translated.replace(' ', '').replace('\n', ''))
            
            if total_chars > 0:
                english_ratio = english_chars / total_chars
                korean_ratio = korean_chars / total_chars
                
                # 영어 비율이 50% 이상이거나 한국어가 10% 미만이면 번역 실패로 간주
                if english_ratio > 0.5 or korean_ratio < 0.1:
                    print(f"번역 실패 (영어:{english_ratio:.1%}, 한국어:{korean_ratio:.1%}): {translated[:50]}...")
                    return text  # 원문 반환
        
        # 빈 결과나 너무 짧은 결과 방지
        if not translated or len(translated.strip()) < 3:
            return text  # 원문 반환
            
        return translated
        
    except Exception as e:
        print(f"번역 오류: {e}")
        return text  # 오류시 원문 반환

# 모델 로드 (실제 사용시 적절한 모델명으로 변경)
print("LLM 모델 로딩 중...")
tokenizer, model = load_translation_model("/workspace/models/exaone-4.0-32b")  # 예시 모델명
# print("실제 사용시 적절한 모델명을 설정하세요.")

LLM 모델 로딩 중...


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

모델 '/workspace/models/exaone-4.0-32b' 로드 완료!


In [3]:
# 데이터 로드 및 구조 확인

# 1. CyberMetric MCQA (영어 전용)
print("=== CyberMetric MCQA 데이터 ===")
cyber_mcqa = pd.read_csv('../data/CyberMetric/mcqa_org.csv')
print(f"데이터 크기: {cyber_mcqa.shape}")
print(f"컬럼: {cyber_mcqa.columns.tolist()}")
print("\n샘플 데이터:")
print(cyber_mcqa.head(2))

print("\n" + "="*50 + "\n")

# 2. SecBench MCQA (영어/중국어 혼용)
print("=== SecBench MCQA 데이터 ===")
sec_mcqa = pd.read_csv('../data/SecBench/mcqa_org.csv')
print(f"데이터 크기: {sec_mcqa.shape}")
print(f"컬럼: {sec_mcqa.columns.tolist()}")
print("\n샘플 데이터:")
print(sec_mcqa.head(2))

# 언어 분포 확인
print("\n언어 분포 분석:")
languages = []
for idx in range(min(100, len(sec_mcqa))):  # 처음 100개만 샘플링
    question = sec_mcqa.iloc[idx]['Question']
    lang = detect_language(question.split('\n')[0])  # 첫 번째 줄만 검사
    languages.append(lang)

from collections import Counter
lang_count = Counter(languages)
print(f"샘플 100개 중 언어 분포: {dict(lang_count)}")

print("\n" + "="*50 + "\n")

# 3. SecBench QA (영어/중국어 혼용)
print("=== SecBench QA 데이터 ===")
sec_qa = pd.read_csv('../data/SecBench/qa_org.csv')
print(f"데이터 크기: {sec_qa.shape}")
print(f"컬럼: {sec_qa.columns.tolist()}")
print("\n샘플 데이터:")
print(sec_qa.head(2))

# 언어 분포 확인
print("\n언어 분포 분석:")
qa_languages = []
for idx in range(min(100, len(sec_qa))):  # 처음 100개만 샘플링
    question = sec_qa.iloc[idx]['Question']
    lang = detect_language(question)
    qa_languages.append(lang)

qa_lang_count = Counter(qa_languages)
print(f"QA 샘플 100개 중 언어 분포: {dict(qa_lang_count)}")

=== CyberMetric MCQA 데이터 ===
데이터 크기: (10180, 2)
컬럼: ['Question', 'Answer']

샘플 데이터:
                                            Question Answer
0  Which of the following refers to the secrecy o...  답변: 4
1  Which type of authentication uses multiple fac...  답변: 3


=== SecBench MCQA 데이터 ===
데이터 크기: (2730, 2)
컬럼: ['Question', 'Answer']

샘플 데이터:
                                            Question Answer
0  以下哪种措施是企业为了遵守数据保护法规，通常会实施的？\n1. 定期进行网络渗透测试\n2....  답변: 2
1  关于个人数据的权益，根据GDPR，数据主体有权：\n1. 随时要求访问、更正或删除他们的个人...  답변: 1

언어 분포 분석:
샘플 100개 중 언어 분포: {'zh': 92, 'en': 8}


=== SecBench QA 데이터 ===
데이터 크기: (270, 2)
컬럼: ['Question', 'Answer']

샘플 데이터:
                   Question                                             Answer
0  为什么现代操作系统的IP ID通常是随机产生的？                               为了提高安全性，防止某些类型的网络攻击。
1   ECA技术如何帮助企业保护其网络免受内部威胁？  ECA技术通过分析内部网络流量的特征，可以识别异常行为和潜在的内部威胁，从而帮助企业加强内部...

언어 분포 분석:
QA 샘플 100개 중 언어 분포: {'zh': 97, 'en': 3}


In [4]:
# MCQA 데이터 번역 함수

def parse_mcqa_question(question_text: str) -> Dict:
    """MCQA 질문을 파싱하여 문제와 선택지를 분리합니다."""
    lines = question_text.strip().split('\n')
    
    # 첫 번째 줄은 문제 
    question = lines[0]
    
    # 나머지는 선택지
    choices = []
    for line in lines[1:]:
        line = line.strip()
        if line and (line.startswith(('1.', '2.', '3.', '4.', '5.', '6.', '7.'))):
            choices.append(line)
    
    return {
        'question': question,
        'choices': choices
    }

def translate_mcqa_entry(question_text: str, answer: str, tokenizer, model, source_lang: str = "auto") -> Tuple[str, str]:
    """MCQA 엔트리를 번역합니다. 언어 자동 감지 지원."""
    
    # 질문 파싱
    parsed = parse_mcqa_question(question_text)
    
    # 질문 번역 (언어 자동 감지)
    if source_lang == "auto":
        detected_lang = detect_language(parsed['question'])
        # print(f"질문 언어 감지: {detected_lang}")  # 로그 줄임
    else:
        detected_lang = source_lang
    
    # 질문 번역
    translated_question = translate_text(parsed['question'], tokenizer, model, detected_lang)
    
    # 각 선택지 번역
    translated_choices = []
    for choice in parsed['choices']:
        # 선택지 번호와 내용 분리
        if '. ' in choice:
            num_part, content = choice.split('. ', 1)
            # 각 선택지마다 언어 감지 (혼용 가능성)
            choice_lang = detect_language(content) if source_lang == "auto" else detected_lang
            translated_content = translate_text(content, tokenizer, model, choice_lang)
            # 번역 결과 정제
            translated_content = clean_translation_result(translated_content)
            translated_choices.append(f"{num_part}. {translated_content}")
        else:
            choice_lang = detect_language(choice) if source_lang == "auto" else detected_lang
            translated_choice = translate_text(choice, tokenizer, model, choice_lang)
            translated_choice = clean_translation_result(translated_choice)
            translated_choices.append(translated_choice)
    
    # 번역된 질문과 선택지 결합 (형식 개선)
    if translated_choices:
        translated_full = translated_question + '\n' + '\n'.join(translated_choices)
    else:
        translated_full = translated_question
    
    # 답변은 그대로 유지 (번호나 간단한 형태)
    translated_answer = answer
    
    return translated_full, translated_answer

def translate_qa_entry(question: str, answer: str, tokenizer, model, source_lang: str = "auto") -> Tuple[str, str]:
    """QA 엔트리를 번역합니다. 언어 자동 감지 지원."""
    
    # 질문과 답변 각각 언어 감지
    if source_lang == "auto":
        q_lang = detect_language(question)
        a_lang = detect_language(answer)
        # print(f"질문 언어: {q_lang}, 답변 언어: {a_lang}")  # 로그 줄임
    else:
        q_lang = a_lang = source_lang
    
    # 번역 및 정제
    translated_question = translate_text(question, tokenizer, model, q_lang)
    translated_question = clean_translation_result(translated_question)
    
    translated_answer = translate_text(answer, tokenizer, model, a_lang)
    translated_answer = clean_translation_result(translated_answer)
    
    return translated_question, translated_answer

print("번역 함수 정의 완료!")

번역 함수 정의 완료!


In [ ]:
# 고속 배치 번역 함수들

def translate_batch_texts(texts: List[str], tokenizer, model, source_lang: str = "en", batch_size: int = 8, save_interval: int = 1000, checkpoint_prefix: str = "checkpoint"):
    """여러 텍스트를 배치로 번역합니다 (중간 저장 기능 포함)."""
    results = []
    checkpoint_file = f"/workspace/fske/data/{checkpoint_prefix}_{source_lang}_progress.json"
    
    # 기존 체크포인트 로드
    start_idx = 0
    if os.path.exists(checkpoint_file):
        try:
            with open(checkpoint_file, 'r', encoding='utf-8') as f:
                checkpoint = json.load(f)
            results = checkpoint.get('results', [])
            start_idx = len(results)
            print(f"📂 체크포인트 발견! {start_idx}개 항목부터 재시작합니다.")
        except Exception as e:
            print(f"체크포인트 로드 오류: {e}, 처음부터 시작합니다.")
    
    print(f"📦 {len(texts)}개 텍스트를 {batch_size}개씩 배치 번역 중... (시작: {start_idx})")
    start_time = time.time()
    
    for i in tqdm(range(start_idx, len(texts), batch_size), desc="🚀 고속 배치 번역"):
        batch_texts = texts[i:i + batch_size]
        batch_results = []
        
        for text in batch_texts:
            try:
                translated = translate_text(text, tokenizer, model, source_lang)
                batch_results.append(translated)
            except Exception as e:
                print(f"배치 번역 오류: {e}")
                batch_results.append(text)  # 오류시 원문 반환
        
        results.extend(batch_results)
        
        # 중간 저장 (save_interval마다)
        if len(results) % save_interval == 0 or len(results) == len(texts):
            checkpoint_data = {
                'results': results,
                'total': len(texts),
                'completed': len(results),
                'timestamp': time.time(),
                'source_lang': source_lang
            }
            try:
                with open(checkpoint_file, 'w', encoding='utf-8') as f:
                    json.dump(checkpoint_data, f, ensure_ascii=False, indent=2)
                print(f"💾 체크포인트 저장: {len(results)}/{len(texts)}개 완료")
            except Exception as e:
                print(f"체크포인트 저장 오류: {e}")
        
        # 진행률 및 속도 표시
        if (i + batch_size) % (save_interval // 10) == 0:
            elapsed = time.time() - start_time
            speed = (len(results) - start_idx) / elapsed if elapsed > 0 else 0
            remaining = (len(texts) - len(results)) / speed if speed > 0 else 0
            print(f"⚡ 진행률: {len(results)}/{len(texts)} ({len(results)/len(texts)*100:.1f}%) | "
                  f"속도: {speed:.1f}개/초 | 남은시간: {remaining/60:.1f}분")
        
        # GPU 메모리 정리
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    # 완료 후 체크포인트 파일 삭제
    if os.path.exists(checkpoint_file):
        os.remove(checkpoint_file)
        print("🗑️ 체크포인트 파일 정리 완료")
    
    total_time = time.time() - start_time
    print(f"🎉 배치 번역 완료! 총 시간: {total_time/60:.1f}분, 평균 속도: {(len(results)-start_idx)/total_time:.1f}개/초")
    
    return results

def translate_cybermetric_mcqa_fast(df: pd.DataFrame, tokenizer, model, batch_size: int = 16):
    """CyberMetric MCQA 데이터를 고속으로 번역합니다."""
    
    print(f"🚀 CyberMetric MCQA 총 {len(df)}개 항목 고속 번역 시작...")
    print(f"💡 배치 크기: {batch_size}, GPU 메모리: {torch.cuda.get_device_properties(0).total_memory/1024**3:.1f}GB")
    
    # 1단계: 데이터 전처리 - 질문과 선택지 분리
    all_questions = []
    all_choices = []
    choice_mapping = []  # 어떤 질문에 속하는지 매핑
    
    for idx, row in tqdm(df.iterrows(), desc="📋 데이터 전처리", total=len(df)):
        question_text = row['Question']
        parsed = parse_mcqa_question(question_text)
        
        # 질문 본문 추가
        all_questions.append(parsed['question'])
        
        # 각 선택지 추가 및 매핑 저장
        choice_start_idx = len(all_choices)
        for choice in parsed['choices']:
            if '. ' in choice:
                num_part, content = choice.split('. ', 1)
                all_choices.append(content)
            else:
                all_choices.append(choice)
        
        choice_mapping.append({
            'start_idx': choice_start_idx,
            'count': len(parsed['choices']),
            'original_choices': parsed['choices']
        })
    
    # 2단계: 배치 번역 실행 (중간 저장 포함)
    print("📝 질문 배치 번역 중...")
    translated_questions = translate_batch_texts(all_questions, tokenizer, model, "en", batch_size, 500, "cybermetric_questions")
    
    print("📋 선택지 배치 번역 중...")
    translated_choices = translate_batch_texts(all_choices, tokenizer, model, "en", batch_size, 1000, "cybermetric_choices")
    
    # 3단계: 결과 재조립
    print("🔧 결과 조립 중...")
    translated_data = []
    
    for idx, row in tqdm(df.iterrows(), desc="🔧 결과 조립", total=len(df)):
        try:
            # 번역된 질문 가져오기
            translated_question = translated_questions[idx]
            
            # 번역된 선택지 조립
            mapping = choice_mapping[idx]
            translated_choice_list = []
            
            for i, original_choice in enumerate(mapping['original_choices']):
                choice_idx = mapping['start_idx'] + i
                
                if '. ' in original_choice:
                    num_part, content = original_choice.split('. ', 1)
                    translated_content = translated_choices[choice_idx]
                    translated_choice_list.append(f"{num_part}. {translated_content}")
                else:
                    translated_choice_list.append(translated_choices[choice_idx])
            
            # 최종 조립
            if translated_choice_list:
                full_question = translated_question + '\\n' + '\\n'.join(translated_choice_list)
            else:
                full_question = translated_question
            
            translated_data.append({
                'Question': full_question,
                'Answer': row['Answer']
            })
            
        except Exception as e:
            print(f"조립 오류 (행 {idx}): {e}")
            translated_data.append({
                'Question': row['Question'], 
                'Answer': row['Answer']
            })
    
    # 최종 결과 중간 저장
    final_checkpoint = {
        'data': translated_data,
        'completed_at': time.time(),
        'total_items': len(df)
    }
    try:
        with open(f"/workspace/fske/data/cybermetric_mcqa_final.json", 'w', encoding='utf-8') as f:
            json.dump(final_checkpoint, f, ensure_ascii=False, indent=2)
        print("💾 최종 결과 저장 완료!")
    except Exception as e:
        print(f"최종 저장 오류: {e}")
    
    print("✅ 고속 번역 완료!")
    return pd.DataFrame(translated_data)

def translate_secbench_mcqa_fast(df: pd.DataFrame, tokenizer, model, batch_size: int = 16):
    """SecBench MCQA 데이터를 고속으로 번역합니다 (영어/중국어 혼용)."""
    
    print(f"🚀 SecBench MCQA 총 {len(df)}개 항목 고속 번역 시작...")
    print(f"💡 배치 크기: {batch_size}, 언어 자동 감지 활성화")
    
    # 1단계: 데이터 전처리 - 질문과 선택지 분리 + 언어 감지
    all_questions = []
    all_choices = []
    choice_mapping = []
    question_langs = []
    choice_langs = []
    
    for idx, row in tqdm(df.iterrows(), desc="📋 데이터 전처리 및 언어 감지", total=len(df)):
        question_text = row['Question']
        parsed = parse_mcqa_question(question_text)
        
        # 질문 언어 감지 및 추가
        q_lang = detect_language(parsed['question'])
        all_questions.append(parsed['question'])
        question_langs.append(q_lang)
        
        # 각 선택지 언어 감지 및 추가
        choice_start_idx = len(all_choices)
        for choice in parsed['choices']:
            if '. ' in choice:
                num_part, content = choice.split('. ', 1)
                c_lang = detect_language(content)
                all_choices.append(content)
                choice_langs.append(c_lang)
            else:
                c_lang = detect_language(choice)
                all_choices.append(choice)
                choice_langs.append(c_lang)
        
        choice_mapping.append({
            'start_idx': choice_start_idx,
            'count': len(parsed['choices']),
            'original_choices': parsed['choices']
        })
    
    # 2단계: 언어별로 그룹화하여 배치 번역
    print("📝 질문 언어별 배치 번역 중...")
    
    # 영어 질문들
    en_questions = [(i, q) for i, (q, lang) in enumerate(zip(all_questions, question_langs)) if lang == 'en']
    zh_questions = [(i, q) for i, (q, lang) in enumerate(zip(all_questions, question_langs)) if lang == 'zh']
    ko_questions = [(i, q) for i, (q, lang) in enumerate(zip(all_questions, question_langs)) if lang == 'ko']
    
    translated_questions = [''] * len(all_questions)
    
    if en_questions:
        print(f"  🇺🇸 영어 질문 {len(en_questions)}개 번역 중...")
        en_texts = [q for _, q in en_questions]
        en_translated = translate_batch_texts(en_texts, tokenizer, model, "en", batch_size)
        for (idx, _), trans in zip(en_questions, en_translated):
            translated_questions[idx] = trans
    
    if zh_questions:
        print(f"  🇨🇳 중국어 질문 {len(zh_questions)}개 번역 중...")
        zh_texts = [q for _, q in zh_questions]
        zh_translated = translate_batch_texts(zh_texts, tokenizer, model, "zh", batch_size)
        for (idx, _), trans in zip(zh_questions, zh_translated):
            translated_questions[idx] = trans
    
    if ko_questions:
        print(f"  🇰🇷 한국어 질문 {len(ko_questions)}개 (번역 생략)...")
        for idx, q in ko_questions:
            translated_questions[idx] = q
    
    # 선택지도 동일하게 처리
    print("📋 선택지 언어별 배치 번역 중...")
    
    en_choices = [(i, c) for i, (c, lang) in enumerate(zip(all_choices, choice_langs)) if lang == 'en']
    zh_choices = [(i, c) for i, (c, lang) in enumerate(zip(all_choices, choice_langs)) if lang == 'zh']
    ko_choices = [(i, c) for i, (c, lang) in enumerate(zip(all_choices, choice_langs)) if lang == 'ko']
    
    translated_choices = [''] * len(all_choices)
    
    if en_choices:
        print(f"  🇺🇸 영어 선택지 {len(en_choices)}개 번역 중...")
        en_texts = [c for _, c in en_choices]
        en_translated = translate_batch_texts(en_texts, tokenizer, model, "en", batch_size)
        for (idx, _), trans in zip(en_choices, en_translated):
            translated_choices[idx] = trans
    
    if zh_choices:
        print(f"  🇨🇳 중국어 선택지 {len(zh_choices)}개 번역 중...")
        zh_texts = [c for _, c in zh_choices]
        zh_translated = translate_batch_texts(zh_texts, tokenizer, model, "zh", batch_size)
        for (idx, _), trans in zip(zh_choices, zh_translated):
            translated_choices[idx] = trans
    
    if ko_choices:
        print(f"  🇰🇷 한국어 선택지 {len(ko_choices)}개 (번역 생략)...")
        for idx, c in ko_choices:
            translated_choices[idx] = c
    
    # 3단계: 결과 재조립
    print("🔧 결과 조립 중...")
    translated_data = []
    
    for idx, row in tqdm(df.iterrows(), desc="🔧 결과 조립", total=len(df)):
        try:
            # 번역된 질문 가져오기
            translated_question = translated_questions[idx]
            
            # 번역된 선택지 조립
            mapping = choice_mapping[idx]
            translated_choice_list = []
            
            for i, original_choice in enumerate(mapping['original_choices']):
                choice_idx = mapping['start_idx'] + i
                
                if '. ' in original_choice:
                    num_part, content = original_choice.split('. ', 1)
                    translated_content = translated_choices[choice_idx]
                    translated_choice_list.append(f"{num_part}. {translated_content}")
                else:
                    translated_choice_list.append(translated_choices[choice_idx])
            
            # 최종 조립
            if translated_choice_list:
                full_question = translated_question + '\\n' + '\\n'.join(translated_choice_list)
            else:
                full_question = translated_question
            
            translated_data.append({
                'Question': full_question,
                'Answer': row['Answer']
            })
            
        except Exception as e:
            print(f"조립 오류 (행 {idx}): {e}")
            translated_data.append({
                'Question': row['Question'], 
                'Answer': row['Answer']
            })
    
    print("✅ SecBench MCQA 고속 번역 완료!")
    return pd.DataFrame(translated_data)

def translate_secbench_qa_fast(df: pd.DataFrame, tokenizer, model, batch_size: int = 16):
    """SecBench QA 데이터를 고속으로 번역합니다 (영어/중국어 혼용)."""
    
    print(f"🚀 SecBench QA 총 {len(df)}개 항목 고속 번역 시작...")
    print(f"💡 배치 크기: {batch_size}, 언어 자동 감지 활성화")
    
    # 1단계: 언어 감지 및 분류
    questions = df['Question'].tolist()
    answers = df['Answer'].tolist()
    
    question_langs = [detect_language(q) for q in tqdm(questions, desc="📋 질문 언어 감지")]
    answer_langs = [detect_language(a) for a in tqdm(answers, desc="📋 답변 언어 감지")]
    
    # 2단계: 언어별 배치 번역
    print("📝 질문 언어별 배치 번역 중...")
    
    # 질문 번역
    en_questions = [(i, q) for i, (q, lang) in enumerate(zip(questions, question_langs)) if lang == 'en']
    zh_questions = [(i, q) for i, (q, lang) in enumerate(zip(questions, question_langs)) if lang == 'zh']
    ko_questions = [(i, q) for i, (q, lang) in enumerate(zip(questions, question_langs)) if lang == 'ko']
    
    translated_questions = [''] * len(questions)
    
    if en_questions:
        print(f"  🇺🇸 영어 질문 {len(en_questions)}개 번역 중...")
        en_texts = [q for _, q in en_questions]
        en_translated = translate_batch_texts(en_texts, tokenizer, model, "en", batch_size)
        for (idx, _), trans in zip(en_questions, en_translated):
            translated_questions[idx] = trans
    
    if zh_questions:
        print(f"  🇨🇳 중국어 질문 {len(zh_questions)}개 번역 중...")
        zh_texts = [q for _, q in zh_questions]
        zh_translated = translate_batch_texts(zh_texts, tokenizer, model, "zh", batch_size)
        for (idx, _), trans in zip(zh_questions, zh_translated):
            translated_questions[idx] = trans
    
    if ko_questions:
        print(f"  🇰🇷 한국어 질문 {len(ko_questions)}개 (번역 생략)...")
        for idx, q in ko_questions:
            translated_questions[idx] = q
    
    # 답변 번역
    print("📋 답변 언어별 배치 번역 중...")
    
    en_answers = [(i, a) for i, (a, lang) in enumerate(zip(answers, answer_langs)) if lang == 'en']
    zh_answers = [(i, a) for i, (a, lang) in enumerate(zip(answers, answer_langs)) if lang == 'zh']
    ko_answers = [(i, a) for i, (a, lang) in enumerate(zip(answers, answer_langs)) if lang == 'ko']
    
    translated_answers = [''] * len(answers)
    
    if en_answers:
        print(f"  🇺🇸 영어 답변 {len(en_answers)}개 번역 중...")
        en_texts = [a for _, a in en_answers]
        en_translated = translate_batch_texts(en_texts, tokenizer, model, "en", batch_size)
        for (idx, _), trans in zip(en_answers, en_translated):
            translated_answers[idx] = trans
    
    if zh_answers:
        print(f"  🇨🇳 중국어 답변 {len(zh_answers)}개 번역 중...")
        zh_texts = [a for _, a in zh_answers]
        zh_translated = translate_batch_texts(zh_texts, tokenizer, model, "zh", batch_size)
        for (idx, _), trans in zip(zh_answers, zh_translated):
            translated_answers[idx] = trans
    
    if ko_answers:
        print(f"  🇰🇷 한국어 답변 {len(ko_answers)}개 (번역 생략)...")
        for idx, a in ko_answers:
            translated_answers[idx] = a
    
    # 3단계: 결과 조립
    print("🔧 결과 조립 중...")
    translated_data = []
    
    for i in range(len(df)):
        translated_data.append({
            'Question': translated_questions[i],
            'Answer': translated_answers[i]
        })
    
    print("✅ SecBench QA 고속 번역 완료!")
    return pd.DataFrame(translated_data)

print("🚀 SecBench 고속 번역 함수 로드 완료!")


🚀 고속 번역 함수 로드 완료!


In [6]:
# 1. CyberMetric MCQA 번역 (영어 → 한국어)

def translate_cybermetric_mcqa(df: pd.DataFrame, tokenizer, model, batch_size: int = 5):
    """CyberMetric MCQA 데이터를 번역합니다."""
    
    translated_data = []
    
    print(f"CyberMetric MCQA 총 {len(df)}개 항목 번역 시작...")
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        try:
            question = row['Question']
            answer = row['Answer']
            
            # 번역 수행 (CyberMetric은 영어만)
            translated_question, translated_answer = translate_mcqa_entry(
                question, answer, tokenizer, model, source_lang="en"
            )
            
            translated_data.append({
                'Question': translated_question,
                'Answer': translated_answer
            })
            
            # 배치마다 잠시 대기 (메모리 관리)
            if (idx + 1) % batch_size == 0:
                time.sleep(1)
                
        except Exception as e:
            print(f"번역 오류 (행 {idx}): {e}")
            # 오류시 원본 데이터 사용
            translated_data.append({
                'Question': row['Question'], 
                'Answer': row['Answer']
            })
    
    return pd.DataFrame(translated_data)

# 실제 번역 실행 (모델이 로드된 경우)
if 'tokenizer' in locals() and 'model' in locals() and tokenizer is not None:
    cyber_mcqa_ko = translate_cybermetric_mcqa_fast(cyber_mcqa, tokenizer, model, batch_size=32)  # 고속 번역
    print("CyberMetric MCQA 번역 완료!")
    print(cyber_mcqa_ko.head(2))
else:
    print("모델이 로드되지 않았습니다. 실제 실행시 모델을 먼저 로드하세요.")


🚀 CyberMetric MCQA 총 10180개 항목 고속 번역 시작...
💡 배치 크기: 32, GPU 메모리: 23.6GB


📋 데이터 전처리: 100%|██████████| 10180/10180 [00:00<00:00, 19742.47it/s]


📝 질문 배치 번역 중...
📦 10180개 텍스트를 32개씩 배치 번역 중...


🚀 고속 배치 번역:   0%|          | 0/319 [00:00<?, ?it/s]

번역 실패 (영어:65.0%, 한국어:25.0%): Maybe "이용 가능성" captures...


🚀 고속 배치 번역:   0%|          | 1/319 [04:21<23:06:24, 261.59s/it]


KeyboardInterrupt: 

In [ ]:
# 2. SecBench MCQA 번역 (중국어 → 한국어)

def translate_secbench_mcqa(df: pd.DataFrame, tokenizer, model, batch_size: int = 5):
    """SecBench MCQA 데이터를 번역합니다."""
    
    translated_data = []
    
    print(f"SecBench MCQA 총 {len(df)}개 항목 번역 시작...")
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        try:
            question = row['Question']
            answer = row['Answer']
            
            # 번역 수행 (SecBench는 영어/중국어 혼용, 자동 감지)
            translated_question, translated_answer = translate_mcqa_entry(
                question, answer, tokenizer, model, source_lang="auto"
            )
            
            translated_data.append({
                'Question': translated_question,
                'Answer': translated_answer
            })
            
            # 배치마다 잠시 대기
            if (idx + 1) % batch_size == 0:
                time.sleep(1)
                
        except Exception as e:
            print(f"번역 오류 (행 {idx}): {e}")
            translated_data.append({
                'Question': row['Question'], 
                'Answer': row['Answer']
            })
    
    return pd.DataFrame(translated_data)

# 실제 번역 실행
if 'tokenizer' in locals() and 'model' in locals() and tokenizer is not None:
    sec_mcqa_ko = translate_secbench_mcqa_fast(sec_mcqa, tokenizer, model, batch_size=32)  # 고속 번역
    print("SecBench MCQA 번역 완료!")
    print(sec_mcqa_ko.head(2))
else:
    print("모델이 로드되지 않았습니다. 실제 실행시 모델을 먼저 로드하세요.")


In [ ]:
# 3. SecBench QA 번역 (중국어 → 한국어)

def translate_secbench_qa(df: pd.DataFrame, tokenizer, model, batch_size: int = 5):
    """SecBench QA 데이터를 번역합니다."""
    
    translated_data = []
    
    print(f"SecBench QA 총 {len(df)}개 항목 번역 시작...")
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        try:
            question = row['Question']
            answer = row['Answer']
            
            # 번역 수행 (SecBench는 영어/중국어 혼용, 자동 감지)
            translated_question, translated_answer = translate_qa_entry(
                question, answer, tokenizer, model, source_lang="auto"
            )
            
            translated_data.append({
                'Question': translated_question,
                'Answer': translated_answer
            })
            
            # 배치마다 잠시 대기
            if (idx + 1) % batch_size == 0:
                time.sleep(1)
                
        except Exception as e:
            print(f"번역 오류 (행 {idx}): {e}")
            translated_data.append({
                'Question': row['Question'], 
                'Answer': row['Answer']
            })
    
    return pd.DataFrame(translated_data)

# 실제 번역 실행
if 'tokenizer' in locals() and 'model' in locals() and tokenizer is not None:
    sec_qa_ko = translate_secbench_qa_fast(sec_qa, tokenizer, model, batch_size=32)  # 고속 번역
    print("SecBench QA 번역 완료!")
    print(sec_qa_ko.head(2))
else:
    print("모델이 로드되지 않았습니다. 실제 실행시 모델을 먼저 로드하세요.")


In [8]:
# 번역 결과 저장

def save_translated_data():
    """번역된 데이터를 CSV 파일로 저장합니다."""
    
    # 저장할 데이터와 경로 정의
    datasets_to_save = [
        {
            'data': 'cyber_mcqa_ko',
            'path': '../data/CyberMetric/mcqa.csv',
            'description': 'CyberMetric MCQA (영어→한국어)'
        },
        {
            'data': 'sec_mcqa_ko', 
            'path': '../data/SecBench/mcqa.csv',
            'description': 'SecBench MCQA (중국어→한국어)'
        },
        {
            'data': 'sec_qa_ko',
            'path': '../data/SecBench/qa.csv', 
            'description': 'SecBench QA (중국어→한국어)'
        }
    ]
    
    for dataset_info in datasets_to_save:
        data_name = dataset_info['data']
        file_path = dataset_info['path']
        description = dataset_info['description']
        
        try:
            # 변수가 존재하는지 확인
            if data_name in locals() or data_name in globals():
                data = locals().get(data_name) or globals().get(data_name)
                
                if data is not None and len(data) > 0:
                    # CSV로 저장
                    data.to_csv(file_path, index=False, encoding='utf-8')
                    print(f"✅ {description} 저장 완료: {file_path} ({len(data)}개 항목)")
                else:
                    print(f"❌ {description}: 데이터가 비어있습니다.")
            else:
                print(f"❌ {description}: 변수 '{data_name}'를 찾을 수 없습니다.")
                
        except Exception as e:
            print(f"❌ {description} 저장 실패: {e}")

# 실제 번역이 완료된 경우에만 저장
print("번역 결과 저장 함수 준비 완료!")
print("실제 번역 완료 후 save_translated_data() 함수를 호출하세요.")

save_translated_data()


번역 결과 저장 함수 준비 완료!
실제 번역 완료 후 save_translated_data() 함수를 호출하세요.
✅ CyberMetric MCQA (영어→한국어) 저장 완료: ../data/CyberMetric/mcqa.csv (10개 항목)
❌ SecBench MCQA (중국어→한국어): 변수 'sec_mcqa_ko'를 찾을 수 없습니다.
❌ SecBench QA (중국어→한국어): 변수 'sec_qa_ko'를 찾을 수 없습니다.
